In [1]:
import wandb

wandb.init(project="baseline-gpu-monitor", name="baseline_run")

wandb: Currently logged in as: ahmadhakimiadnan (ahmadhakimiadnan-other) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [2]:
#import model

In [3]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from datasets import load_dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct",
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
print("model is loaded")

2025-08-22 10:37:02.692170: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-22 10:37:02.698371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755830222.705586  489504 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755830222.707742  489504 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-22 10:37:02.715550: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model is loaded


In [4]:
#load dataset

In [5]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

df = pd.read_csv("CMNEE.csv")
df = df[df['instruction'].notnull() & df['new_output'].notnull()]

if 'input' not in df.columns:
    df['input'] = ''

train_df, val_df = train_test_split(df, test_size=0.15, random_state=42)

data = DatasetDict({
    "train": Dataset.from_pandas(train_df, preserve_index=False),
    "validation": Dataset.from_pandas(val_df, preserve_index=False),
    "test": Dataset.from_pandas(val_df.copy(), preserve_index=False)  # duplicate
})

def create_prompt(example):
    prompt = f"Instruction:\n{example['instruction']}\n\n"
    if example['input']:
        prompt += f"Input:\n{example['input']}\n\n"
    example["prompt"] = prompt
    return example

data = data.map(create_prompt)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=512
    )

tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/2547 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/2547 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

In [6]:
#visualize chatbot

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr

base_model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
def build_prompt(instruction, input_text=""):
    prompt = f"Instruction:\n{instruction.strip()}\n"
    if input_text.strip():
        prompt += f"\nInput:\n{input_text.strip()}\n"
    prompt += "\nResponse:\n"
    return prompt

def generate_text(instruction, input_text):
    prompt = build_prompt(instruction, input_text)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True
        )
    # Only return the part after "Response:"
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return full_output.split("Response:")[-1].strip()

interface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(label="Instruction", lines=2, placeholder="e.g. What is the best team in London?"),
        gr.Textbox(label="Input (optional)", lines=2, placeholder="Add context if needed"),
    ],
    outputs=gr.Textbox(label="Model Response"),
    title="Llama 3.2 1B Baseline Model",
    description="Interact with the baseline LLaMA 3.2 1B model without fine-tuning."
)

interface.launch(share=True)

2025-08-10 16:12:02.471483: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-10 16:12:02.477926: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754813522.484444 1158947 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754813522.486225 1158947 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-10 16:12:02.492730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://175dcf96e41792713e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [1]:
#evaluate baseline model

In [2]:
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import torch
import logging
from transformers import logging as hf_logging

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import nltk
nltk.download('punkt')

base_model = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model)
base = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16, device_map="auto")

logging.getLogger().setLevel(logging.ERROR)
hf_logging.set_verbosity_error()

#evaluation
true_answers = []
predicted_answers = []

for sample in tqdm(data["test"]):
    instruction = sample["instruction"]
    input_text = sample["input"]
    true_output = sample["new_output"]

    prompt = f"Instruction:\n{instruction.strip()}\n"
    if input_text is not None and input_text.strip():
        prompt += f"\nInput:\n{input_text.strip()}\n"
    prompt += "\nResponse:\n"

    inputs = tokenizer(prompt, return_tensors="pt").to(base.device)

    with torch.no_grad():
        output_ids = base.generate(
            **inputs,
            max_new_tokens=128,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=False
        )
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predicted_response = generated_text.split("Response:")[-1].strip()

    predicted_answers.append(predicted_response)
    true_answers.append(true_output.strip())
    
#f1
true_token_sets = [set(ans.lower().split()) for ans in true_answers]
pred_token_sets = [set(ans.lower().split()) for ans in predicted_answers]

all_tokens = list(set().union(*true_token_sets, *pred_token_sets))
mlb = MultiLabelBinarizer(classes=all_tokens)
y_true_bin = mlb.fit_transform(true_token_sets)
y_pred_bin = mlb.transform(pred_token_sets)

f1 = f1_score(y_true_bin, y_pred_bin, average="macro")
print(f"\nMacro F1 Score on validation set: {f1:.4f}")

#BERTScore
from bert_score import score as bert_score
print("\nCalculating BERTScore...")

P = predicted_answers
R = true_answers

P_scores, R_scores, F1_scores = bert_score(P, R, lang='en', verbose=True)
average_bert_f1 = F1_scores.mean().item()
print(f"Average BERTScore F1 on validation set: {average_bert_f1:.4f}")

#BLEU
print("\nCalculating BLEU Score...")

smoothie = SmoothingFunction().method4
bleu_scores = []
for ref, pred in zip(true_answers, predicted_answers):
    ref_tokens = [nltk.word_tokenize(ref.lower())]
    pred_tokens = nltk.word_tokenize(pred.lower())
    score = sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)
    bleu_scores.append(score)

average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score on validation set: {average_bleu:.4f}")

#ROUGE
print("\nCalculating ROUGE Scores...")

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge1_list, rouge2_list, rougeL_list = [], [], []

for ref, pred in zip(true_answers, predicted_answers):
    scores = scorer.score(ref, pred)
    rouge1_list.append(scores['rouge1'].fmeasure)
    rouge2_list.append(scores['rouge2'].fmeasure)
    rougeL_list.append(scores['rougeL'].fmeasure)

avg_rouge1 = sum(rouge1_list) / len(rouge1_list)
avg_rouge2 = sum(rouge2_list) / len(rouge2_list)
avg_rougeL = sum(rougeL_list) / len(rougeL_list)

print(f"Average ROUGE-1 F1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2 F1: {avg_rouge2:.4f}")
print(f"Average ROUGE-L F1: {avg_rougeL:.4f}")

2025-08-19 12:27:33.452696: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-19 12:27:33.459302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755577653.466064 3318871 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755577653.467800 3318871 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-19 12:27:33.474359: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 450/450 [10:07<00:00,  1.35s/it]



Macro F1 Score on validation set: 0.0914

Calculating BERTScore...
calculating scores...
computing bert embedding.


  0%|          | 0/14 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 1.36 seconds, 331.40 sentences/sec
Average BERTScore F1 on validation set: 0.8444

Calculating BLEU Score...
Average BLEU Score on validation set: 0.0561

Calculating ROUGE Scores...
Average ROUGE-1 F1: 0.1836
Average ROUGE-2 F1: 0.1207
Average ROUGE-L F1: 0.1825
